# Step 4: Evaluation (AI-as-a-Judge)

마지막 단계에서는 우리가 만든 AI 수의사들의 실력을 평가합니다.
사람이 직접 채점하는 대신, 또 다른 AI(LLM)에게 채점 기준표를 주고 평가를 맡기는 **Model-based Evaluation** 방식을 실습합니다.

In [1]:
import os
import pandas as pd
import json
from openai import OpenAI

# OpenAI 클라이언트 설정 (API Key 하드코딩)
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"\n
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## 1. 정답 및 결과 로드
5개 케이스 각각에 대한 정답과 에이전트들의 결과를 불러옵니다.

In [2]:
# 정답 데이터 로드
df_truth = pd.read_csv("./data/processed_sample.csv")
print(f"Loaded {len(df_truth)} ground truth cases.")

# 결과 로드 함수
def load_result(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    return ""

# 모든 결과를 리스트로 로드
single_results = []
multi_results = []

for i in range(len(df_truth)):
    single_results.append(load_result(f"./data/single_result_{i}.md"))
    multi_results.append(load_result(f"./data/multi_result_{i}.md"))

Loaded 5 ground truth cases.


## 2. 평가 함수 작성
채점관 AI에게 줄 '채점 기준표'를 작성합니다.
정확성(Accuracy), 구체성(Completeness), 공감능력(Empathy) 세 가지 항목으로 평가하도록 지시합니다.

In [3]:
def evaluate_answer(answer, truth, problem_desc, criteria):
    prompt = f"""
    당신은 수의학 전문 평가자입니다.
    실제 수의사의 답변(Ground Truth)과 문제 상황(Problem Description)을 바탕으로 학생(AI)의 답변을 평가하세요.
    
    문제 상황 (Problem Description):
    {problem_desc}
    
    실제 수의사 정답 (Ground Truth):
    {truth}
    
    학생 AI 답변:
    {answer}
    
    평가 기준:
    {criteria}
    
    평가 결과는 반드시 아래와 같은 정확한 구조의 JSON 객체로 제공해야 합니다:
    {{
      "Accuracy": {{"score": 0, "reasoning": "text"}},
      "Completeness": {{"score": 0, "reasoning": "text"}},
      "Empathy": {{"score": 0, "reasoning": "text"}}
    }}
    (0을 0-10 사이의 점수로, text를 한국어로 작성된 점수 부여 사유로 교체하세요).
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"Evaluation failed: {e}")
        return None



## 3. 평가 실행 (반복)
모든 케이스에 대해 점수를 매기고 평균을 냅니다.

In [4]:
criteria = """
1. Accuracy (정확성): 의학적 조언이 올바르고 안전한가?
2. Completeness (완전성): 긴급도, 원인, 가정 내 대처법을 모두 다루었는가?
3. Empathy (공감능력): 걱정하는 보호자에게 적절하고 친절한 어조인가?
"""

all_single_scores = []
all_multi_scores = []
all_vet_scores = []

for i in range(len(df_truth)):
    print(f"Evaluating Case #{i+1}...")
    ground_truth = df_truth.iloc[i]['answer']
    problem_desc = df_truth.iloc[i]['formatted_prompt']
    
    s_score = evaluate_answer(single_results[i], ground_truth, problem_desc, criteria)
    m_score = evaluate_answer(multi_results[i], ground_truth, problem_desc, criteria)
    v_score = evaluate_answer(ground_truth, ground_truth, problem_desc, criteria)
    
    if s_score: all_single_scores.append(s_score)
    if m_score: all_multi_scores.append(m_score)
    if v_score: all_vet_scores.append(v_score)

# 결과를 DataFrame으로 정리하여 평균 계산
def get_avg_scores(scores_list):
    if not scores_list: return {}
    avg = {'Accuracy': 0, 'Completeness': 0, 'Empathy': 0}
    for s in scores_list:
        avg['Accuracy'] += s['Accuracy']['score']
        avg['Completeness'] += s['Completeness']['score']
        avg['Empathy'] += s['Empathy']['score']
    for k in avg:
        avg[k] /= len(scores_list)
    return avg

avg_single = get_avg_scores(all_single_scores)
avg_multi = get_avg_scores(all_multi_scores)
avg_vet = get_avg_scores(all_vet_scores)

print("\n--- Average Single Agent Scores ---")
print(avg_single)
print("\n--- Average Multi Agent Scores ---")
print(avg_multi)
print("\n--- Average Vet (Ground Truth) Scores ---")
print(avg_vet)


Evaluating Case #1...
Evaluating Case #2...
Evaluating Case #3...
Evaluating Case #4...
Evaluating Case #5...

--- Average Single Agent Scores ---
{'Accuracy': 9.6, 'Completeness': 9.6, 'Empathy': 9.6}

--- Average Multi Agent Scores ---
{'Accuracy': 5.8, 'Completeness': 5.6, 'Empathy': 6.0}

--- Average Vet (Ground Truth) Scores ---
{'Accuracy': 9.4, 'Completeness': 8.6, 'Empathy': 9.4}


In [ ]:
# 시각화
import matplotlib.pyplot as plt

if avg_single and avg_multi and avg_vet:
    df_scores = pd.DataFrame({
        'Metric': list(avg_single.keys()),
        'Single Agent': list(avg_single.values()),
        'Multi Agent': list(avg_multi.values()),
        'Vet (Ground Truth)': list(avg_vet.values())
    })

    df_scores.set_index('Metric').plot(kind='bar', ylim=(0, 10), rot=0)
    plt.title('Performance Comparison (Avg of 5 Cases)')
    plt.ylabel('Score (0-10)')
    plt.show()
